In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go




In [ ]:
# Use the U.S. ticker FOR LIFE 360 INC (LIF)
# FIND DAILY PRICE AND PERCENT CHANGE
ticker = yf.Ticker("LIF")

info = ticker.info
current_price = info.get("currentPrice")
previous_close = info.get("previousClose")
percent_change = ((current_price - previous_close) / previous_close) * 100

print(f"Current Price: ${current_price:.2f}")
print(f"Change: {percent_change:.2f}%")



Current Price: $97.68
Change: -0.19%


In [ ]:
# Extract key metrics
metrics = {
    "Market Cap": info.get("marketCap"),
    "P/E Ratio": info.get("trailingPE"),
    "EPS (TTM)": info.get("trailingEps"),
    "Revenue (TTM)": info.get("totalRevenue"),
    "Last Updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

df_metrics = pd.DataFrame(metrics, index=["Life360"])
display(df_metrics)

df_metrics.to_csv("../data/processed/lif_metrics.csv", index=False)

,Market Cap,P/E Ratio,EPS (TTM),Revenue (TTM),Last Updated
Life360,7574680576,257.04343,0.38,427399008,2025-10-28 11:39:57


In [ ]:
#YTD PERFORMANCE DATA
# Get 1 year of daily data
hist = ticker.history(period="1y")

# 52-week stats
high_52 = hist["High"].max()
low_52 = hist["Low"].min()

# YTD performance
start_of_year = pd.Timestamp(datetime(datetime.now().year, 1, 1), tz='UTC')
ytd_data = hist.loc[hist.index >= pd.to_datetime(start_of_year)]
if not ytd_data.empty:
    start_price = ytd_data["Close"].iloc[0]
    current_price = ytd_data["Close"].iloc[-1]
    ytd_change = ((current_price - start_price) / start_price) * 100
else:
    ytd_change = None

df_ytd = pd.DataFrame({
    "Metric": ["52-Week High", "52-Week Low", "YTD Performance"],
    "Value": [
        f"${high_52:.2f}",
        f"${low_52:.2f}",
        f"{ytd_change:.2f}%" if ytd_change else "N/A"
    ]
})

display(df_ytd)



,Metric,Value
0,52-Week High,$112.54
1,52-Week Low,$29.62
2,YTD Performance,133.31%


In [13]:
# Stock Price Chart
# Do we include this here or straight in Tableau?

def get_stock_data(ticker_symbol, period):
    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(period=period)
    hist.reset_index(inplace=True)
    hist['Date'] = pd.to_datetime(hist['Date'])
    return hist

def plot_stock_chart(hist, title="Life360 (LIF)"):
    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(
        x=hist['Date'],
        open=hist['Open'],
        high=hist['High'],
        low=hist['Low'],
        close=hist['Close'],
        name='Price'
    ))

    fig.update_layout(
        title=title,
        yaxis_title="Price (USD)",
        xaxis_title="Date",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        hovermode="x unified",
        height=500
    )

    fig.show()



In [15]:
# Define periods
period_options = {
    "1D": "1d",
    "1W": "5d",
    "1M": "1mo",
    "6M": "6mo",
    "1Y": "1y",
    "5Y": "5y"
}

# Fetch all period data once
data_cache = {label: get_stock_data("LIF", period) for label, period in period_options.items()}

# Create initial figure (1Y default)
initial_period = "1Y"
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=data_cache[initial_period]['Date'],
    open=data_cache[initial_period]['Open'],
    high=data_cache[initial_period]['High'],
    low=data_cache[initial_period]['Low'],
    close=data_cache[initial_period]['Close'],
    name="LIF"
))

# Add dropdown buttons
fig.update_layout(
    title="Life360 (LIF) — Interactive Stock Chart",
    template="plotly_dark",
    yaxis_title="Price (USD)",
    xaxis_title="Date",
    xaxis_rangeslider_visible=False,
    height=500,
    updatemenus=[
        dict(
            buttons=[
                dict(label=label,
                     method="update",
                     args=[
                         {"x": [data_cache[label]['Date']],
                          "open": [data_cache[label]['Open']],
                          "high": [data_cache[label]['High']],
                          "low": [data_cache[label]['Low']],
                          "close": [data_cache[label]['Close']]},
                         {"title": f"Life360 (LIFX) — {label} View"}
                     ])
                for label in period_options.keys()
            ],
            direction="down",
            showactive=True,
            x=0.15,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ]
)

fig.show()


In [16]:
fig.write_html("../data/processed/lifx_chart.html")

It's too complex... I'm going to make this more simpler and understandable.

In [17]:
def fetch_hist(symbol: str, period: str, interval: str):
    t = yf.Ticker(symbol)
    df = t.history(period=period, interval=interval)
    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"])
    return df